In [1]:
## I generated a mapnode to run tractography on multiple ROI's in a single node
## I think it's going to be easier to just look at the output and copy the results myself
## vs trying to do this in a datasink node...
import glob, os, sys, shutil
from  os.path import join as opj

nipype_results_dir = '/data/NipypeScratch/runpbx2_gpu_dtispace_fixedwarps/'
tractographyResultsData = "_subject_id_%s/pbx2/mapflow/"

subjList = [x.replace('_subject_id_','') for x in os.listdir(nipype_results_dir) if '_subject_id' in x]

In [2]:
print(len(subjList),"Subject Directories located")

(194, 'Subject Directories located')


In [4]:
import re
import shutil

for s in subjList:
    try:
        mapResultDirs = os.listdir(opj(nipype_results_dir,tractographyResultsData%s))
    except:
        print("Missing data for %s" % s)
        next
    
    ### Now loop through the mapNode results
    ## I am looking for the fdt_paths.nii.gz and command.txt
    for mr in mapResultDirs:
        fdt_paths = opj(nipype_results_dir,tractographyResultsData%s,mr,'fdt_paths.nii.gz')
        command = opj(nipype_results_dir,tractographyResultsData%s,mr,'command.txt')
        waytotal = opj(nipype_results_dir,tractographyResultsData%s,mr,'waytotal')
        
        
        ## tpreviously it was _wimt it is now _trans when I am running hthis pipeline
        #print(fdt_paths)
        if os.path.isfile(fdt_paths) and os.path.isfile(command) and os.path.isfile(waytotal):
            with open(command,'r') as fp:
                line = fp.readline()
                m  = re.search('seed=/data/.*/DTI_ROIs\/(?P<seedMask>.*)_trans.nii.gz',line)
                if m:
                    fdtTgtDirName = opj('/data/HCP_BedpostData/addlInfoV2/subject/',s,'pbxResults','DTI')
                    if not os.path.isdir(fdtTgtDirName):
                        os.makedirs(fdtTgtDirName)
                    
                    fdtTgtFile = opj(fdtTgtDirName,m.groups()[0]+'_fdt_paths.nii.gz')
                    if not os.path.isfile(fdtTgtFile):
                        shutil.copy(fdt_paths,fdtTgtFile)
                        
                    fdtWayTotalFile = opj(fdtTgtDirName,m.groups()[0]+'_waytotal')
                    if not os.path.isfile(fdtWayTotalFile):
                        shutil.copy(waytotal,fdtWayTotalFile)   

Missing data for 196144
Missing data for 198855
Missing data for 199150
